In [3]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ (всех)

import pandas as pd

# Загружаем нужные столбцы
cols = ['Номер из NAV','Адрес магазина','дата планового открытия','статус','ИФ кольца','ИФ КОЛЬЦА ОБРУЧ',
        'ИФ печатки','ИФ серьги','ИФ подвес культ','ИФ подвес декор','БК кольца','БК кольца обруч','БК печатки',
        'БК серьги','БК подвес культ','БК подвес декор','ИТОГО, нормы в штуках, без накоплений']

df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols)

# Убираем ненужную инфу
df.rename({'ИТОГО, нормы в штуках, без накоплений':"sum_all"}, axis=1, inplace=True)
df = df[df['sum_all'] > 0]
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружен'])]

# Добавляем сумму по ИФ\БК и оставляем этот столбец вместо столбцов с нормами по иф\бк
sum_cols = [x for x in cols if 'ИФ' in x or 'БК' in x ]
df['sum_gold'] = df[sum_cols].sum(axis=1)
df.drop(axis=1, columns=sum_cols, inplace=True)

# Количество открытых и к открытию всего и только по золоту
shops_count = pd.DataFrame({
    'Магов открыто': [df.sum_all[df.статус !='склад'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус !='склад')].count()],
    'Магов к открытию': [df.sum_all[df.статус =='склад'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус =='склад')].count()],
    'Итого магов': [df.sum_all.count(),df.sum_gold[df.sum_gold > 0].count()]})

# Список новых магов с датами
new_shops = df[['Номер из NAV', 'Адрес магазина', 'дата планового открытия']][(df.sum_gold > 0) & (df.статус =='склад')]
new_shops.sort_values('дата планового открытия', inplace=True)

# Нормы ИФ/БК по ТПГ
row_number = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])

skip_row = row_number.ЦОК.tolist()
skip_row_n = skip_row.index('подгруппа')-1

norm_all = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)

lst = norm_all.columns.tolist()

cols = (
    [lst[lst.index('Название из NAV')]] +
    lst[lst.index(1001):lst.index(1102)+1] +
    lst[lst.index(1501):lst.index(1597)+1]
)
norm_all = norm_all[cols]
gold_shops = df[['Номер из NAV', 'статус']][df.sum_gold > 0]
merged = pd.merge(gold_shops, norm_all, left_on='Номер из NAV', right_on='Название из NAV', how = 'left')

del merged['Название из NAV'], merged['Номер из NAV']

# Все нецифры меняю на ноль. из ексель подтягивается значок '-'
nostatus = [i for i in merged.columns.to_list() if i !='статус']
merged[nostatus] = merged[nostatus].apply(pd.to_numeric, errors='coerce').fillna(0)


merged = merged.groupby(by='статус').sum()
merged = merged.T
merged['sum_all'] = merged.sum(axis=1)
merged = merged[['открыт', 'sum_all']]

# СОХРАНЯЕМ ВСЕ В 1 ФАЙЛ

with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\для_новые_нормы.xlsx'
) as writer:
    shops_count.to_excel(writer, sheet_name='количество', index=False)
    new_shops.to_excel(writer, sheet_name='новые_маги', index=False)
    merged.to_excel(writer, sheet_name='новые_нормы')

In [24]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ (всех)

import pandas as pd
import numpy as np

# Загружаем нужные столбцы
cols = ['Номер из NAV','Адрес магазина','дата планового открытия','статус','ИФ кольца','ИФ КОЛЬЦА ОБРУЧ',
        'ИФ печатки','ИФ серьги','ИФ подвес культ','ИФ подвес декор','БК кольца','БК кольца обруч','БК печатки',
        'БК серьги','БК подвес культ','БК подвес декор','ИТОГО, нормы в штуках, без накоплений']

df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols)

# Убираем ненужную инфу
df.rename({'ИТОГО, нормы в штуках, без накоплений':"sum_all"}, axis=1, inplace=True)
df = df[df['sum_all'] > 0]
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружен'])]


# Добавляем сумму по ИФ\БК и оставляем этот столбец вместо столбцов с нормами по иф\бк
sum_cols = [x for x in cols if 'ИФ' in x or 'БК' in x ]
df['sum_gold'] = df[sum_cols].sum(axis=1)
df.drop(axis=1, columns=sum_cols, inplace=True)

# Количество открытых и к открытию всего и только по золоту
shops_count = pd.DataFrame({
    'Магов открыто': [df.sum_all[df.статус !='склад'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус !='склад')].count()],
    'Магов к открытию': [df.sum_all[df.статус =='склад'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус =='склад')].count()],
    'Итого магов': [df.sum_all.count(),df.sum_gold[df.sum_gold > 0].count()]})

# Список новых магов с датами
new_shops = df[['Номер из NAV', 'Адрес магазина', 'дата планового открытия']][(df.sum_gold > 0) & (df.статус =='склад')]
new_shops.sort_values('дата планового открытия', inplace=True)

# Нормы ИФ/БК по ТПГ
row_number = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])

skip_row = row_number.ЦОК.tolist()
skip_row_n = skip_row.index('подгруппа')-1

norm_all = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)

lst = norm_all.columns.tolist()

cols = (
    [lst[lst.index('Название из NAV')]] +
    lst[lst.index(1001):lst.index(1102)+1] +
    lst[lst.index(1501):lst.index(1597)+1]
)
norm_all = norm_all[cols]
gold_shops = df[['Номер из NAV', 'статус']][df.sum_gold > 0]
merged = pd.merge(gold_shops, norm_all, left_on='Номер из NAV', right_on='Название из NAV', how = 'left')

In [26]:
del merged['Название из NAV'], merged['Номер из NAV']



In [29]:
merged.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx',
index=False)

In [42]:

merged.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx',
index=False)

In [41]:
merged.iloc[:,10:]

,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,...,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597
0,38.052312,49.952869,11.056456,89.155474,108.967801,49.530819,6.39167,1.96542,1.96542,35.844099,...,0.538855,24.000000,36.000000,12.000000,2.236082,0.486937,0.196516,0,0,0
1,13.542316,17.777567,3.934847,31.729257,38.780203,17.627365,2.27471,0.699466,0.699466,12.756442,...,0.197130,13.137387,13.137387,6.568693,0.917651,0.199831,0.080647,0,0,0
2,22.127212,29.047320,6.429269,51.843420,63.364180,28.801900,3.71672,1.14288,1.14288,20.843147,...,0.296785,12.000000,12.000000,12.000000,1.175572,0.255997,0.103314,0,0,0
3,30.880175,40.537703,8.972524,72.351362,88.429442,40.195201,5.18696,1.59497,1.59497,29.088168,...,0.405522,24.000000,12.000000,12.000000,1.588056,0.345821,0.139565,0,0,0
4,28.860322,37.886157,8.385637,67.618905,82.645328,37.566058,4.84768,1.49065,1.49065,27.185529,...,0.395090,24.000000,24.000000,12.000000,1.749936,0.381072,0.153792,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,15.477636,20.318144,4.497172,36.263657,44.322247,20.146476,0,0,0,14.579454,...,0.252717,12.000000,12.000000,0.000000,0.899083,0.195787,0.079015,0,0,0
941,17.416186,22.862960,5.060436,40.805624,49.873540,22.669791,0,0,0,16.405508,...,0.225624,12.000000,12.000000,12.000000,1.025924,0.223409,0.090163,0,0,0
942,15.523211,20.377972,4.510414,36.370438,44.452758,20.205799,0,0,0,14.622384,...,0.231807,12.000000,12.000000,0.000000,0.855110,0.186212,0.075151,0,0,0
943,9.956432,13.070227,2.892935,23.327634,28.511553,12.959797,0.436598,0.134253,0.134253,9.378651,...,0.145380,8.217849,7.607455,4.555486,0.651011,0.146829,0.057350,0,0,0


In [15]:

# Нормы ИФ/БК по ТПГ
row_number = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])

skip_row = row_number.ЦОК.tolist()
skip_row_n = skip_row.index('подгруппа')-1

norm_all = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)

lst = norm_all.columns.tolist()

cols = (
    [lst[lst.index('Название из NAV')]] +
    lst[lst.index(1001):lst.index(1102)+1] +
    lst[lst.index(1501):lst.index(1597)+1]
)
norm_all = norm_all[cols]



In [21]:
norm_all

,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597
729,20.039732,12.511408,2.498024,13.976773,6.071701,4.216331,14.019665,12.267207,8.762291,12.427710,...,0.180161,12.000000,12.000000,0.000000,0.746502,0.162561,0.065606,0,0,0
1170,36.081517,22.526777,4.497689,25.165167,10.932091,7.591500,25.242394,22.087094,15.776496,22.376079,...,0.271443,24.000000,12.000000,12.000000,1.306096,0.284420,0.114785,0,0,0
1278,31.806412,19.857701,3.964782,22.183481,9.636806,6.692024,22.251558,19.470113,13.907224,19.724858,...,0.302340,12.000000,12.000000,12.000000,1.187255,0.258541,0.104341,0,0,0
1590,28.095877,17.541102,3.502251,19.595557,8.512577,5.911333,19.655692,17.198730,12.284807,17.423756,...,0.254414,14.381236,13.313046,7.972101,1.139270,0.256950,0.100363,0,0,0


In [20]:
norm_all[norm_all['Название из NAV'].isin([4531, 4908, 3626, 4422])]

KeyError: 'Название из NAV'

In [22]:





merged = norm_all.T
merged['sum_all'] = merged.sum(axis=1)


# СОХРАНЯЕМ ВСЕ В 1 ФАЙЛ

with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\для_новые_нормы.xlsx'
) as writer:
    
    merged.to_excel(writer, sheet_name='новые_нормы')

In [2]:
#ДОСТАЕТ НОРМЫ ПО БУКВАМ И ЗОДИАКАМ 

import pandas as pd
import numpy as np

# Читаем столбцы с нормами по буквам и зодиакам из реестра
cols = [2, *range(368,374), *range(469,475)]
link = r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm'    
norm = pd.read_excel(link, sheet_name='НОРМЫ 17.12', skiprows=4, usecols=cols)

# Обрезаем пустые строки, делим на буквы и зодиаки
last_row = norm.index[norm['Название из NAV'].isna()][0]-1
norm = norm.loc[:last_row]
norm.set_index('Название из NAV',inplace=True)

b_columns = [1088, 1089, 1090, 1586, 1587, 1588]
z_columns = [1091, 1092, 1093, 1589, 1590, 1591]
norm_b = norm[b_columns]
norm_z = norm[z_columns]
norm_b['sum_if'], norm_b['sum_bk'] = norm_b[[1088, 1089, 1090]].sum(axis=1), norm_b[[1586, 1587, 1588]].sum(axis=1)
norm_z['sum_if'], norm_z['sum_bk'] = norm_z[[1091, 1092, 1093]].sum(axis=1), norm_z[[1589, 1590, 1591]].sum(axis=1)

with pd.ExcelWriter (r'C:\Остатки\norm_bz.xlsx') as writer:
    norm_b.to_excel(writer, sheet_name='norm_b')
    norm_z.to_excel(writer, sheet_name='norm_z')

C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
#ВЫДАЕТ СПИСОК АКТУАЛЬНЫХ МАГОВ ИФ\БК С НОРМАМИ ПО ТГ И ДАТЫ ОТКРЫТИЯ

import pandas as pd
import numpy as np

#выгружаем датафрэйм с актуальными нормами ИФ БК из реестра магов 
cols_df = ['Номер из NAV', 'Адрес магазина', 'дата планового открытия','статус', 'ИФ кольца', 'ИФ КОЛЬЦА ОБРУЧ', 'ИФ печатки', 'ИФ серьги', 
        'ИФ подвес культ', 'ИФ подвес декор', 'БК кольца', 'БК кольца обруч', 'БК печатки', 'БК серьги', 
        'БК подвес культ', 'БК подвес декор']
      
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols_df)

#выбираем открытые маги с суммой норм по иф/бк больше нуля
df = df[df.статус.str.contains('откр|отгр|склад', na=False, case=False)]
df['сумма'] = df.iloc[:, 3:].sum(axis=1)
df = df[df['сумма'] > 0]
df = df[df['Номер из NAV'] != 3323]# Тамбов носовская


all_shops = df.drop(columns=['статус','сумма','дата планового открытия'])
all_shops = all_shops.rename(columns={'Номер из NAV':'Код'})
all_shops['Код'] = all_shops['Код'].astype(np.int64)

# делим на два отдельных иф и бк
cols = all_shops.columns.tolist()
shops_if = all_shops[[*cols[:5],cols[7],cols[6],cols[5]]]
shops_bk = all_shops[[*cols[:2], *cols[8:11], cols[13],cols[12],cols[11]]]

#выгружаем из файла "наполняемость" актуальную структуру магов иф/бк, добавляет туда нормы и новые маги из датафрэйма выше
shops_if_list = pd.read_excel(
    r'C:\Остатки\наполняемость с 10012020.xlsx',
    sheet_name='ИФ',skiprows=3, usecols=[2],keep_default_na=False)

shops_bk_list = pd.read_excel(
    r'C:\Остатки\наполняемость с 10012020.xlsx',
    sheet_name='БК',skiprows=3, usecols=[2],keep_default_na=False)

# список новых магов которые надо добавить
add_to_if = pd.DataFrame({'Код':[x for x in shops_if['Код'].tolist() if x not in shops_if_list['Код'].tolist()]})
add_to_bk = pd.DataFrame({'Код':[x for x in shops_bk['Код'].tolist() if x not in shops_bk_list['Код'].tolist()]})

# добавляем новые маги в структуру
shops_if_list = shops_if_list.append(add_to_if, sort=False)
shops_bk_list = shops_bk_list.append(add_to_bk, sort=False)

#соединяем "структуру" и нормы
shops_if = pd.merge(shops_if_list,shops_if,how='left',on='Код')
shops_bk = pd.merge(shops_bk_list,shops_bk,how='left',on='Код')


#удаляем индекс и сохраняем на отдельные вкладки
shops_if.set_index('Код', drop=True,inplace=True)
shops_bk.set_index('Код', drop=True,inplace=True)

#создаем датафрейм из магов с датами открытия последние 90 дней
from datetime import date, timedelta

three_month_ago = date.today() - timedelta(90)
df['дата планового открытия'] = df['дата планового открытия'].apply(lambda x:x.date())#чтобы ошибка не выпадала о формате
dates = df[['Номер из NAV', 'дата планового открытия']][df['дата планового открытия'] > three_month_ago]
dates.set_index('Номер из NAV', drop=True,inplace=True)

#сохраняем все в эксель
with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\актуальные маги.xlsx'
) as writer:
    shops_if.to_excel(writer, sheet_name='ИФ')
    shops_bk.to_excel(writer, sheet_name='БК')
    dates.to_excel(writer, sheet_name='даты')

In [1]:
# СПИСОК НОРМ ПО ТГ

import pandas as pd
import numpy as np

#читает файл с реестром
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)

#вынимает нужные две строки и сохраняет
rep_tn = df.iloc[0:1,28:74]
rep_tn.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx',
index=False)

In [1]:
#СЧИТАЕТ КОЛИЧЕСТВО МАГАЗОВ 3 МЕСЯЦА ДО СЕГОДНЯ И ПОСЛЕ

import pandas as pd
import numpy as np

df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=([2, 3, 6, 18] + list(range(28, 41))), parse_dates=[6] )

# выбираем маги с нужным статусом, с нормами по тн больше нуля 
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружено'])]
df[['дата планового открытия']] = df['дата планового открытия'].fillna(df['дата планового открытия'].min())
df['sum'] = df.iloc[ : , 5:].sum(axis=1)
df = df[df['sum']>0]


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 947 entries, 1 to 1875
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Номер из NAV             947 non-null    float64       
 1   Адрес магазина           947 non-null    object        
 2   дата планового открытия  947 non-null    datetime64[ns]
 3   статус                   947 non-null    object        
 4   ИФ кольца                947 non-null    float64       
 5   ИФ КОЛЬЦА ОБРУЧ          947 non-null    float64       
 6   ИФ печатки               947 non-null    float64       
 7   ИФ серьги                947 non-null    float64       
 8   ИФ подвес культ          947 non-null    float64       
 9   ИФ подвес декор          947 non-null    float64       
 10  ИФ аксессуары            947 non-null    float64       
 11  БК кольца                947 non-null    float64       
 12  БК кольца обруч          947 non-nu

In [3]:
df[['дата планового открытия']][df['дата планового открытия'] > '2020-07-01'].sort_values(['дата планового открытия'])


,дата планового открытия
947,2020-07-12
951,2020-08-01
950,2020-08-10
954,2020-08-10
952,2020-08-11
953,2020-08-25
968,2020-09-30
971,2020-10-01
967,2020-10-01
966,2020-10-01


In [ ]:
from ipywidgets import DatePicker
DPicker = DatePicker()
display(DPicker)

def date_value(date):
    
    return date

DPicker.observe(date_value, names='value')
print(date)

In [4]:
dPicker.observe()

TypeError: observe() missing 1 required positional argument: 'handler'

In [1]:
from IPython.display import display
from ipywidgets import DatePicker

dPicker = DatePicker()

def on_button_clicked(b):
    print(b['old'], '->', b['new'])

dPicker.observe(on_button_clicked, names='value')
display(dPicker)

DatePicker(value=None)

None -> 2024-07-02


In [5]:
import os
import pandas as pd


def read_and_save(file_name):
    
    path = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Назарова А.С/бланки_заказов/'
    path_read = path + file_name
    path_save = path + 'готовые/' + file_name
    df = pd.read_excel(path_read, skiprows=16, skipfooter=1)
    df = df.iloc[3:]

    #transpose df with sizes as columns
    sizes = df.set_index(['Артикул', 'Размер'])['Кол-во, шт.'].unstack()
    sizes.reset_index(inplace=True)

    # groupby df on article, add columns and regroup as needed
    group_cols = ['Артикул', 'Наименование', 'Проба', 'цвет металла', '%угара по металлу', 'Цена работы руб./гр.' ]
    df_grouped = df.groupby(group_cols)[['Кол-во, шт.','Общий вес, гр.']].sum()
    df_grouped.reset_index(inplace=True)
    df_grouped['Ср. вес., гр.'] = df_grouped['Общий вес, гр.']/df_grouped['Кол-во, шт.']
    df_grouped = df_grouped[df_grouped.columns[[0, 1, 2, 3, 4, 6, 8, 7, 5]]]

    df_grouped.merge(sizes, on='Артикул').to_excel(path_save, index=False)
    

directory = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Назарова А.С/бланки_заказов'

# delete old files
for x in os.listdir(directory + '/готовые'):
    os.remove(directory + '/готовые/' + x)
    
# create new files    
for x in os.listdir(directory):
    if 'xls' in x:
        read_and_save(x)    


KeyError: 'Общий вес, гр.'